# Development Right Transfers
#### Meeting Agenda: Development Rights Transfer & Conversion Analysis

1. Objectives & Data Review
    * Goal: Standardize ETL of transfer → conversion process & confirm data integration
    * Review analysis requirements
    * Review of data sources:
        * LT Info TDR Transactions (APN, type, land capability, quantity)
        * Parcel Master (jurisdiction, town center proximity, local plan)
        * Accela (transfer status, permit data)
2. Coding Plan & Standardization
    * Walkthrough of data integration approach
    * Confirm transfer first, then convert process
    * Address any inconsistencies or edge cases
3. Key Analyses
    * Land Capability: SEZ, sensitive, non-sensitive
    * Distance from Center: Trends by proximity
    * Interjurisdictional Activity: Transfers & conversions across boundaries
4. Next Steps
    * Assign action items & timeline for completion


## Notes: From Ken
- Transfer Reporting Status - comes from LTinfo
    - Transfers come out of LTinfo 
- Status of Transaction - comes from Accela
    - transaction is considered complete and development rights are moved to recieving parcel when the transfer is acknowledged
- Status of the Development on the Recieving Parcel
    - associate the transaction in LTinfo to the development project in Accela/Local Jurisdiction data
    - what is the status of the development project? (i.e. when is it existing on the ground)
 
- Transfer vs Conversion sequence
    - should be transfer dev rights then convert on the recieving parcel 
- Parcel Geneology Lookup needs to be built
    - Identify old APNs and current APNs
- Data Clean-up
    - categorization of unit types has evolved (e.g. PRUU vs RUU) same/same now
- Conversions
    - we track the transfers and then convert onto the recieving parcel (or onsite conversion)
    - track conversion net change
- 

## Setup

### Packages

In [1]:
import pandas as pd
import os
import pathlib
import arcpy
from arcgis.features import FeatureLayer, GeoAccessor, GeoSeriesAccessor
from utils import *
from datetime import datetime
from time import strftime  
# set data frame display options
# pandas options
pd.options.mode.copy_on_write = True
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows    = 999
pd.options.display.float_format = '{:,.2f}'.format
   
# set environement workspace to in memory 
arcpy.env.workspace = 'memory'
# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)
arcpy.env.outputCoordinateSystem = sr

# current working directory
local_path = pathlib.Path().absolute()
# set data path as a subfolder of the current working directory TravelDemandModel\2022\
data_dir   = local_path.parents[0] / 'Reporting/data/raw_data'
# folder to save processed data
out_dir    = local_path.parents[0] / 'Reporting/data/processed_data'
# local geodatabase path
local_gdb = Path("C:\GIS\Scratch.gdb")
# network path to connection files
filePath = "F:/GIS/PARCELUPDATE/Workspace/"
# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")
sdeEdit    = os.path.join(filePath, "Edit.sde")

## Data Processing
### Data Pipeline Overview
1. Extract data from LT Info, Parcel Master, and Accela.
2. Clean and preprocess data for consistency.
3. Merge datasets using APN as the primary key.
4. Standardize workflow: **transfer first, then convert**.
5. Identify and resolve inconsistencies.

### 1. Extract data from LT Info, Parcel Master, and Accela.

#### Data Sources
- **LT Info TDR Transactions**: Tracks APN, development right type, land capability, and quantity.
- **Parcel Master**: Provides jurisdiction, town center proximity, and
- **Accela**: Contains transfer status and permit details.

> Sources
* https://www.laketahoeinfo.org/WebServices/List
* https://maps.trpa.org/server/rest/services/
* https://parcels.laketahoeinfo.org/TdrTransaction/TransactionList
* sdeBase, sdeCollect, sdeTabular

#### 1.1 Parcel Master

In [ ]:
# web service and database paths
# portal_ParcelMaster = 'https://maps.trpa.org/server/rest/services/Parcel_Master/FeatureServer/0'
sde_ParcelMaster    = Path(sdeBase) / "sde.SDE.Parcels\\sde.SDE.Parcel_Master"
# get spatially enabled dataframes
sdfParcels = pd.DataFrame.spatial.from_featureclass(sde_ParcelMaster)

#### 1.2 LTInfo Data

In [ ]:
# transfer grid downloaded from LTinfo https://parcels.laketahoeinfo.org/TdrTransaction/TransactionList
# dfTransactionsGrid = pd.read_csv(local_path / "data/raw_data/TransactedAndBankedDevelopmentRights.csv")

# grid path
# dfTransactionsGrid = pd.read_excel(local_path / "data/raw_data/TdrTransactions as of 02_06_2025 12_00 PM.xlsx")a
dfTransfers   = pd.read_excel(data_dir / "TdrTransactions as of 02_06_2025 12_00 PM.xlsx", sheet_name='Transfers')
dfConversions = pd.read_excel(data_dir / "TdrTransactions as of 02_06_2025 12_00 PM.xlsx", sheet_name='Conversions') 
dfConvTransfer = pd.read_excel(data_dir / "TdrTransactions as of 02_06_2025 12_00 PM.xlsx", sheet_name='Conversion with Transfers')

In [ ]:
## LT Info Data
# get banked
dfDevRightBanked     = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# Verified Development Rights from Accela as a DataFrame
# Development Rights Transacted and Banked as a DataFrame
dfDevRightTransacted = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfDevRightForAccela  = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelDevelopmentRightsForAccela/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# All Parcels as a DataFrame
dfLTParcel           = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

#### 1.3 Accela Permit Data

In [ ]:
# API access to download excel file of Accela Record Details
accelaRecorDetails = "https://laketahoeinfo.org/Api/GetAccelaRecordDetailsExcel/1A77D078-B83E-44E0-8CA5-8D7429E1A6B4"
# download the file
dfAccelaRecord = pd.read_excel(accelaRecorDetails)

In [ ]:
# get detailed project data report
dfDetailedProjectData = pd.read_excel(data_dir / "PermitStatusReport.xlsx")
dfDetailedProjectData.info()

### 3. Merge & Transform

In [ ]:
final_schema = ['Transaction Status',
                'Transaction Type',
                'Development Right',
                'Sending Parcel APN',
                'Receiving Parcel APN',
                'Sending Quantity',
                'Receiving Quantity',
                'Sending Bailey Rating',
                'Receiving Bailey Rating',
                'Issued',
                'Acknowledged',
                'Project Completed', 
                'APN',
                'JURISDICTION',  
                'PLAN_TYPE',
                'LOCATION_TO_TOWNCENTER',
                'SHAPE']

In [ ]:
# filter columns in sdf Parcels
parcels = sdfParcels[['APN', 'JURISDICTION', 'PLAN_TYPE', 'LOCATION_TO_TOWNCENTER', 'SHAPE']]
# merge dfTransfers with dfDetailedProjectData
df = pd.merge(dfTransfers, dfDetailedProjectData, left_on='Accela ID', right_on='File Number', how='left')
# merge Sending APN to Parcel APN
df = pd.merge(parcels, df, left_on='APN', right_on= 'Sending Parcel APN', how='inner')
# limit to final schema columns
df = df[final_schema]
# convert numeric columns to float
df['Sending Quantity'] = df['Sending Quantity'].astype(float)
df['Receiving Quantity'] = df['Receiving Quantity'].astype(float)
# export df to feature class
df.spatial.to_featureclass(local_gdb / "Parcel_Transfers", sanitize_columns=True, overwrite=True)

In [ ]:
# let's do this join twice once for the sending and once for the receiving
# merge dfTransfers with dfDetailedProjectData
df = pd.merge(dfTransfers, dfDetailedProjectData, left_on='Accela ID', right_on='File Number', how='left')

# merge Receiving APN to Parcel APN
dfRecieving = pd.merge(parcels, df, left_on='APN', right_on= 'Receiving Parcel APN', how='inner')
dfSending   = pd.merge(parcels, df, left_on='APN', right_on= 'Sending Parcel APN', how='inner')

# limit to final schema columns
dfRecieving = dfRecieving[final_schema]
dfSending   = dfSending[final_schema]

dfSending['Transaction Type'] = 'Sending'
dfRecieving['Transaction Type'] = 'Receiving'
dfSending['Net_Change'] = 0 - dfSending['Sending Quantity']
dfRecieving['Net_Change'] = dfRecieving['Receiving Quantity']

# group by APN, Development Right Type, and sum net change
dfRecieving = dfRecieving.groupby(['APN', 'Development Right'], as_index=False).agg({'Net_Change': 'sum'})
dfSending   = dfSending.groupby(['APN', 'Development Right'], as_index=False).agg({'Net_Change': 'sum'})

# stack the two dataframes
df = pd.concat([dfRecieving, dfSending], axis=0, ignore_index=True)
# df.spatial.to_featureclass(local_gdb / "Parcel_Transfers", sanitize_columns=True, overwrite=True)

In [ ]:
# merge dfTransfers with dfDetailedProjectData
df = pd.merge(dfTransfers, dfDetailedProjectData, left_on='Accela ID', right_on='File Number', how='left')
# merge Sending APN to Parcel APN
df = pd.merge(df, parcels, left_on='Sending Parcel APN', right_on='APN', how='left')
# rename parcels fields with prefix SENDING_
df.rename(columns={'JURISDICTION': 'SENDING_JURISDICTION', 'PLAN_TYPE': 'SENDING_PLAN_TYPE', 'LOCATION_TO_TOWNCENTER': 'SENDING_LOCATION_TO_TOWNCENTER'}, inplace=True)
# merge Receiving APN to Parcel APN
df = pd.merge(df, parcels, left_on='Receiving Parcel APN', right_on='APN', how='left')
# rename parcels fields with prefix RECEIVING_
df.rename(columns={'JURISDICTION': 'RECEIVING_JURISDICTION', 'PLAN_TYPE': 'RECEIVING_PLAN_TYPE', 'LOCATION_TO_TOWNCENTER': 'RECEIVING_LOCATION_TO_TOWNCENTER'}, inplace=True)

## Key Analyses & Insights

### Land Capability Analysis
- Categorize transfers by SEZ, sensitive, and non-sensitive land.

In [3]:
# web service and database paths
# portal_ParcelMaster = 'https://maps.trpa.org/server/rest/services/Parcel_Master/FeatureServer/0'
sde_ParcelMaster    = Path(sdeBase) / "sde.SDE.Parcels\\sde.SDE.Parcel_Master"
# get spatially enabled dataframes
sdfParcels = pd.DataFrame.spatial.from_featureclass(sde_ParcelMaster)

# Development Rights Transacted and Banked as a DataFrame
dfDevRightTransacted = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")


c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\pandas\core\dtypes\cast.py:1066: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\pandas\core\dtypes\cast.py:1091: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [4]:
sdeParcelHistory = Path(sdeBase) / "sde.SDE.Parcels\\sde.SDE.Parcel_History"
parcel_history = pd.DataFrame.spatial.from_featureclass(sdeParcelHistory)

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\pandas\core\dtypes\cast.py:1066: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\Lib\site-packages\pandas\core\dtypes\cast.py:1091: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [5]:
parcel_history.head()

,OBJECTID,APN,PPNO,Status,IsActive,APN_Current,APNS_CURRENT,Current_Address,LandUse_Description,b2006Active,b2007Active,b2008Active,b2009Active,b2010Active,b2011Active,b2012Active,b2013Active,b2014Active,b2015Active,b2016Active,b2017Active,b2018Active,b2019Active,b2020Active,b2021Active,Years_Active,GlobalID,created_user,created_date,last_edited_user,last_edited_date,b2022Active,b2023Active,JURISDICTION,b2024Active,SHAPE
0,1,1318-26-101-073,"131,826,101,073.00",Active,1,1318-26-101-073,1318-26-101-073,121 DAGGETT WY,Single Family Residential,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"2024,2023,2022,2021,2020,2019,2018,2017,2016,2...",{6AA51D3D-D4BE-40B9-B200-D24CF526435A},<NA>,NaT,SDE,2024-11-06 21:04:17,1,1,DG,1,"{""x"": 766213.2640147405, ""y"": 4317531.93414231..."
1,2,1318-25-110-009,"131,825,110,015.00",Inactive,0,1318-25-110-015,1318-25-110-015,145 EASY ST,Single Family Residential,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"2022,2021,2020,2019,2018,2017,2016,2015,2014,2...",{EB53A69A-5450-4059-B5CE-B46FEA952BE6},<NA>,NaT,SDE,2024-11-06 21:04:17,1,0,DG,0,"{""x"": 767245.3864448238, ""y"": 4317492.34174883..."
2,3,1319-30-532-005,"131,930,532,005.00",Active,1,1319-30-532-005,1319-30-532-005,331 QUAKING ASPEN LN #C,Single Family Residential,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"2024,2023,2022,2021,2020,2019,2018,2017,2016,2...",{F1DD4A30-89C6-40F5-93EB-1B14F1C6E42D},<NA>,NaT,SDE,2024-11-06 21:04:17,1,1,DG,1,"{""x"": 769891.8772091372, ""y"": 4317518.66204449..."
3,4,1318-10-417-048,"131,810,417,048.00",Active,1,1318-10-417-048,1318-10-417-048,626 LAKEVIEW DR,Multi-Family Residential,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"2024,2023,2022,2021,2020,2019,2018,2017,2016,2...",{ECE3B8FC-3DE0-457F-BB0D-A5E0FB99EDFE},<NA>,NaT,SDE,2024-11-06 21:04:17,1,1,DG,1,"{""x"": 763818.0650090575, ""y"": 4321022.56166765..."
4,5,1318-10-416-036,"131,810,416,036.00",Active,1,1318-10-416-036,1318-10-416-036,612 ALMA WY,Single Family Residential,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,"2024,2023,2022,2021,2020,2019,2018,2017,2016,2...",{97444EB1-F47B-432F-906A-D93822AF055D},<NA>,NaT,SDE,2024-11-06 21:04:17,1,1,DG,1,"{""x"": 764303.8076290907, ""y"": 4320935.60598197..."


In [6]:
def get_new_apn(old_apn, parcel_history):
    """
    Given an old APN, return the new APN(s) from the parcel history DataFrame.

    Parameters:
    - old_apn (str): The historical APN to look up.
    - parcel_history (pd.DataFrame): DataFrame with parcel history, containing APN, Status, APN_Current, and APNs_Current.

    Returns:
    - str | list | None: The new APN (str), list of APNs if split, or None if not found.
    """
    row = parcel_history[parcel_history['APN'] == old_apn]

    if row.empty:
        return None

    row = row.iloc[0]  # There should be only one match per old APN

    # Priority to APN_Current if it's a clean one-to-one mapping
    if pd.notna(row['APN_Current']):
        return row['APN_Current']

    # If there's a list in APNs_Current
    if pd.notna(row['APNs_Current']):
        apns = [apn.strip() for apn in row['APNs_Current'].split(',')]
        return apns if len(apns) > 1 else apns[0]

    return None

In [7]:
get_new_apn('022-343-27', parcel_history)

'022-343-027'

In [ ]:
# get a copy
df = dfDevRightTransacted[['APN',
                        'RecordType',
                        'DevelopmentRight',
                        'LandCapability',
                        'IPESScore',
                        'CumulativeBankedQuantity',
                        'RemainingBankedQuantity',
                        'LastUpdated',
                        'TransactionNumber',
                        'TransactionApprovalDate',
                        'SendingParcel',
                        'ReceivingParcel',
                        'AccelaID',
                        'JurisdictionPermitNumber']].copy()
# filter columns of copy
parcels = sdfParcels[['APN', 
                        'JURISDICTION',  
                        'PLAN_ID',
                        'PLAN_NAME',
                        'ZONING_ID',
                        'ZONING_DESCRIPTION',
                        'TOWN_CENTER',
                        'LOCATION_TO_TOWNCENTER',
                        'TAZ', 
                        'WITHIN_BONUSUNIT_BNDY', 
                        'WITHIN_TRPA_BNDY',
                        'LOCAL_PLAN_HYPERLINK',
                        'LTINFO_HYPERLINK',
                        'PARCEL_ACRES', 'PARCEL_SQFT', 'SHAPE']].copy()

# fix issues with LOCATION_TO_TOWNCENTER values
parcels['LOCATION_TO_TOWNCENTER'] = parcels['LOCATION_TO_TOWNCENTER'].str.strip()
parcels['LOCATION_TO_TOWNCENTER'].replace({'Within Quarter Mile of Town Center': 'Quarter Mile Buffer',
                                            'Within Town Center': 'Town Center',
                                            'Further than Quarter Mile from Town Center': 'Outside Buffer'
                                            }, inplace=True)

# filter to only include the record types in the list
record_types = ['Conversion With Transfer Receiving Parcel',
                'Conversion With Transfer Sending Parcel',
                'Transfer Receiving Parcel', 
                'Transfer Sending Parcel']

# filter the dataframe to only include the record types in the list
df = df[df['RecordType'].isin(record_types)]
# # # if ApprovalData is not  then it is approved
df = df[df['TransactionApprovalDate'] != '']

# categorize bailey rating
landcap_dict = {'1b':'SEZ',
                '1a':'Sensitive',
                '1c':'Sensitive',
                '2':'Sensitive',
                '3':'Sensitive',
                '4':'Non-Sensitive',
                '5':'Non-Sensitive',
                '6':'Non-Sensitive',
                '7':'Non-Sensitive'}

# land capability class = strip -1 if string starts with Bailey
df.loc[df['LandCapability'].notnull() & df['LandCapability'].str.startswith('Bailey'), 
                                        'LandCapability'] = df['LandCapability'].str.strip('Bailey ')

# map the land capability class to the dictionary after filtering out 'IPES'
df.loc[df['LandCapability'].notnull() & df['LandCapability'] != 'IPES', 
                                        'LandCapabilityCategory'] = df['LandCapability'].map(landcap_dict)

# land capability category 
# if IPESScore class to 0 = SEZ, 1-725 = Sensitive, >725 = Non Sensitive
df.loc[df['IPESScore'] == 0, 'LandCapabilityCategory'] = 'SEZ'
df.loc[(df['IPESScore'] > 0) & (df['IPESScore'] <= 725), 'LandCapabilityCategory'] = 'Sensitive'
df.loc[df['IPESScore'] > 725, 'LandCapabilityCategory'] = 'Non-Sensitive'

# merge with parcels
df = pd.merge(df, parcels, left_on='APN', right_on='APN', how='left')
# SendingVsReceiving from RecordType
def classify_sending_receiving(record_type):
    if "Receiving Parcel" in record_type:
        return "Receiving"
    elif "Sending Parcel" in record_type:
        return "Sending"
    return "Unknown"

# get the sensitivity of the counterpart parcel
def get_counterpart_sensitivity(row):
    if row['SendingVsReceiving'] == 'Receiving':
        return apn_to_sensitivity.get(row['SendingParcel'], 'Unknown')
    elif row['SendingVsReceiving'] == 'Sending':
        return apn_to_sensitivity.get(row['ReceivingParcel'], 'Unknown')
    return 'Unknown'

# Build the From → To sensitivity string
def classify_sensitivity_transition(row):
    if row['SendingVsReceiving'] == 'Sending':
        return f"From {row['LandCapabilityCategory']} to {row['CounterpartSensitivity']}"
    elif row['SendingVsReceiving'] == 'Receiving':
        return f"From {row['CounterpartSensitivity']} to {row['LandCapabilityCategory']}"
    return 'Unknown'

# Look up counterparty Town Center classification
def get_counterpart_towncenter(row):
    if row['SendingVsReceiving'] == 'Receiving':
        return apn_to_towncenter.get(row['SendingParcel'], 'Unknown')
    elif row['SendingVsReceiving'] == 'Sending':
        return apn_to_towncenter.get(row['ReceivingParcel'], 'Unknown')
    return 'Unknown'

# Build the From → To Town Center transition string
def classify_towncenter_transition(row):
    if row['SendingVsReceiving'] == 'Sending':
        return f"From {row['LOCATION_TO_TOWNCENTER']} to {row['CounterpartTownCenter']}"
    elif row['SendingVsReceiving'] == 'Receiving':
        return f"From {row['CounterpartTownCenter']} to {row['LOCATION_TO_TOWNCENTER']}"
    return 'Unknown'

# classify sending vs receiving
df['SendingVsReceiving'] = df['RecordType'].apply(classify_sending_receiving)
# Create lookup from APN to LandCapabilityCategory
apn_to_sensitivity = df.set_index('APN')['LandCapabilityCategory'].to_dict()
df['CounterpartSensitivity'] = df.apply(get_counterpart_sensitivity, axis=1)
df['Sensitivity_Transition'] = df.apply(classify_sensitivity_transition, axis=1)
# Create a lookup from APN to LOCATION_TO_TOWNCENTER
apn_to_towncenter = df.set_index('APN')['LOCATION_TO_TOWNCENTER'].to_dict()
df['CounterpartTownCenter'] = df.apply(get_counterpart_towncenter, axis=1)
df['TownCenter_Transition'] = df.apply(classify_towncenter_transition, axis=1)

# Build combined category for land sensitivity and town center location
def build_land_towncenter_combo(row):
    sending_sens = row['LandCapabilityCategory']  # e.g. 'Sensitive' or 'Non-Sensitive'
    sending_loc = row['LOCATION_TO_TOWNCENTER']  # e.g. 'Town Center', 'Quarter Mile Buffer', 'Outside Buffer'
    receiving_sens = row['CounterpartSensitivity']  # e.g. 'Sensitive' or 'Non-Sensitive'
    receiving_loc = row['CounterpartTownCenter']  # e.g. 'Town Center', 'Quarter Mile Buffer', 'Outside Buffer'
    
    if pd.isna(sending_sens) or pd.isna(sending_loc) or pd.isna(receiving_sens) or pd.isna(receiving_loc):
        return 'Unknown'
    return f"Sending: {sending_sens} ({sending_loc}) → Receiving: {receiving_sens} ({receiving_loc})"

df['LandSensitivity_and_TownCenter_Transition'] = df.apply(build_land_towncenter_combo, axis=1)

# export to feature class
df.spatial.to_featureclass(local_gdb / "Parcel_Transfers", sanitize_columns=False, overwrite=True)

### Proximity Analysis
- Assess distance of transfers from town centers.

In [ ]:
df = df.copy()
# categorize
landcap_dict = {'1b':'SEZ',
                '1a':'Sensitive',
                '2':'Sensitive',
                '3':'Sensitive',
                '4':'Non-Sensitive',
                '5':'Non-Sensitive',
                '6':'Non-Sensitive',
                '7':'Non-Sensitive'}
# map land capability to land capability category
df['Sending_Land_Capability_Category'] = df['Sending Bailey Rating'].map(landcap_dict)
# map land capability to land capability category
df['Receiving_Land_Capability_Category'] = df['Receiving Bailey Rating'].map(landcap_dict)


In [ ]:
# groupby sending_land_capability_category and receiving_land_capability_category
df_landcap_group = df.groupby(['Sending_Land_Capability_Category', 'Receiving_Land_Capability_Category', 'Development Right']).agg({
                                'Sending Quantity':'sum',
                                'Receiving Quantity': 'sum'}).reset_index()
df_landcap_group

In [ ]:
df_towncenter = df.groupby(['SENDING_LOCATION_TO_TOWNCENTER', 'RECEIVING_LOCATION_TO_TOWNCENTER', 'Development Right']).agg({
                                'Sending Quantity':'sum',
                                'Receiving Quantity': 'sum'}).reset_index() 
df_towncenter

In [ ]:
# sending land capability category vs receiving location to town center

# recieving land capability category vs sending location to town center



### Interjurisdictional Transfers
- Examine development right transfers across jurisdictional boundaries.

In [ ]:
# 
df_jurisdiction = df.groupby(['SENDING_JURISDICTION', 'RECEIVING_JURISDICTION', 'Development Right']).agg({
                                'Sending Quantity':'sum',
                                'Receiving Quantity': 'sum'}).reset_index() 
df_jurisdiction

## Next Steps
### Action Items
- Build Accela Report that gets us issued data 
- Get Accela ID and Jurisdiction Permit Number into LTinfo Web Service Development Rights Transacted and Banked
- Fix Parcel geneology for current 'APN', 'Recieving APN' and 'Sending APN' 